In [ ]:
import os 
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import Model
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
# from metrics2 import *
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
# from utility_methods2 import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio
import tensorflow.keras.backend as K
import math

## Load data

In [ ]:
id_name=ID_DS_LIST[2] # selects the ID dataset.
id_model=ID_MODEL_LIST[2]  # select the deep model used for training ID dataset.
print(id_name,id_model)

(org_traing_data, org_training_labels),(id_eva_data,org_testing_labels)  = load_dataset(id_name)
org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

x_train = org_traing_data_processed
y_train = org_training_labels
x_test = id_eva_data_processed
y_test = org_testing_labels

In [ ]:
normal_class = 5

x_train_normal = x_train[y_train ==normal_class]
y_train_normal = y_train[y_train==normal_class]

x_train_anomaly = x_train[y_train!=normal_class]
y_train_anomaly= y_train[y_train!=normal_class]

x_test_normal = x_test[y_test==normal_class]
y_test_normal = y_test[y_test==normal_class]

x_test_anomaly = x_test[y_test!=normal_class]
y_test_anomaly = y_test[y_test!=normal_class]

## Declare model 

In [ ]:
class MemoryBank(layers.Layer):
    def __init__(self, mem_dim, fea_dim):
        super(MemoryBank, self).__init__()
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim

        memory_weight_init = tf.random_normal_initializer()
        self.memory_weight = tf.Variable(
            initial_value=memory_weight_init(shape=(self.mem_dim, self.fea_dim), dtype="float32"),
            trainable=True,
        )
        
    def cosine_sim(self, a, b):
        num = tf.linalg.matmul(a, tf.transpose(b, perm=[1, 0]))
        denom_a =  tf.math.sqrt(tf.math.reduce_sum(a**2,axis=1,keepdims=True))
        denom_b = tf.math.sqrt(tf.math.reduce_sum(b**2,axis=1,keepdims=True))
        denom =  tf.linalg.matmul(denom_a,tf.transpose(denom_b))
        w = (num + 1e-12) / (denom + 1e-12)
        return w   

    def sparse_shink(self,x,mem_dim):
        # lamb = 1 / mem_dim # may need to be tuned!!!
        lamb = 0.0025
        num = tf.keras.activations.relu(x - lamb)*x
        denom = tf.abs(x - lamb) + 1e-12
        wh = (num + 1e-12) / denom 
        wh_renorm = wh/tf.norm(wh,ord=1,axis=1,keepdims=True)
        return wh_renorm 
        
    def call(self, z):
        d = self.cosine_sim(z,self.memory_weight)
        w = tf.nn.softmax(d)
        wh_renorm = self.sparse_shink(w,self.mem_dim)
        z_hat = tf.linalg.matmul(wh_renorm,self.memory_weight)
        return z_hat, wh_renorm

In [ ]:
class MemAE(keras.Model):
    def __init__(self, **kwargs):
        super(MemAE, self).__init__(**kwargs)
        self.encoder = self.getEncoder()
        self.decoder = self.getDecoder()
        
    def getEncoder(self):
        encoder_inputs = keras.Input(shape=(32, 32, 3))
        # x = keras.layers.Flatten()(encoder_inputs)
        x = layers.Conv2D(32, 3, strides=2, padding='same', activation=tf.nn.relu)(encoder_inputs)
        x = layers.Conv2D(64, 3, strides=2, padding='same', activation=tf.nn.relu)(x)
        x = layers.Conv2D(128, 3, strides=2, padding='same', activation=tf.nn.relu)(x)
        x = keras.layers.Flatten()(x)
        z_hat, wh_renorm = MemoryBank(800,2048)(x)
        encoder = keras.Model(encoder_inputs, [z_hat, wh_renorm], name="encoder")
        return encoder 
        
    def getDecoder(self):
        latent_inputs = keras.Input(shape=(2048,))
        x = layers.Reshape(target_shape=(4, 4, 128))(latent_inputs)
        
        x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation=tf.nn.relu)(x)
        x = layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation=tf.nn.relu)(x)
        x = layers.Conv2DTranspose(3, 3, strides=2, padding='same', activation='sigmoid')(x)
        decoder_outputs = keras.layers.Reshape(target_shape=(32, 32, 3))(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        return decoder
        
    def train_step(self, x):
    
        with tf.GradientTape() as tape:
            z_hat, wh_renorm = self.encoder(x)
            x_hat = self.decoder(z_hat)

            mse = tf.reduce_sum(tf.square(x - x_hat))
            mem_etrp = tf.reduce_sum((-wh_renorm) * tf.math.log(wh_renorm + 1e-12))
            loss = tf.reduce_mean(mse + (0.0002 * mem_etrp))
            
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {"loss": loss}
    
    def test_step(self, data):
        x, y = data[0],data[1]
        z_hat, wh_renorm = self.encoder(x)
        x_hat = self.decoder(z_hat)

        mse = tf.reduce_sum(tf.square(x - x_hat))
        mem_etrp = tf.reduce_sum((-wh_renorm) * tf.math.log(wh_renorm + 1e-12))
        loss = tf.reduce_mean(mse + (0.0002 * mem_etrp))
        return {"loss": loss}
    
    def call(self, inputs):
        z_hat, w_hat = self.encoder(inputs)
        x_hat = self.decoder(z_hat)
        return x_hat, w_hat

## Train model 

In [ ]:
for i in range(10):
    model = MemAE()
    #model.compile(optimizer=keras.optimizers.Adam(),run_eagerly=True)
    model.compile(optimizer=keras.optimizers.Adam())
    # model.compile(loss='mean_squared_error',optimizer='rmsprop')
    normal_class = i
    x_train_normal = x_train[y_train ==normal_class]
    x_test_normal = x_test[y_test==normal_class]
    model.fit(x_train_normal,validation_data=(x_test_normal,x_test_normal), epochs=50, batch_size=256)
    
    # model.save('./MemAE_models/%s_%s_%s.h5'%(id_name,id_model,i))
    model.save_weights('./MemAE_models/%s_%s_weights_%s.h5'%(id_name,id_model,i))

## Test model

### In sample

In [ ]:
# for i in range(10):
#     a,_ = model(x_test_anomaly[i].reshape(1,28,28,1))
#     plt.imshow(x_test_anomaly[i,:,:],'gray')
#     plt.show() 
#     plt.imshow(a[0,:,:],'gray')
#     plt.show()


In [ ]:
def mark(x_1, x_2):
    diff = np.abs(x_1 - x_2)
    marks = np.mean(np.power(diff, 2), axis=(1,2,3))
    return marks

In [ ]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.vstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [ ]:
testimg = np.load('./datas/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)
sc_id = [[] for i in range(10)]
for c in range(10):
    normal_class = c 
    
    new_model = MemAE()
    x, _ = new_model(x_test[0].reshape(1,32,32,3))
    new_model.load_weights('./MemAE_models/%s_%s_weights_%s.h5'%(id_name,id_model,normal_class))
    model = new_model 
    
    x_test_normal = testimg[normal_class]
    for i in range(len(x_test_normal)):
        xx,_ = model(x_test_normal[i].reshape(1,32,32,3))
        score = mark(x_test_normal[i],xx)
        sc_id[c].append(-score)
        
np.save('./MemAE_datas_OOD/testimgs_%s_%s.npy'%(id_model,id_name),sc_id)

In [ ]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN


In [ ]:
testimg = np.load('./datas/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)
sc_id = np.load('./MemAE_datas_OOD/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)

for i, ds_name in enumerate(OOD_DS_LIST):
    ood = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('ClassesDatas/'+ood+'.npy',allow_pickle=True)
    lists = []
    for i in range(10):
        if (len(das[i]) != 0):
            lists.append(i)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    FP = dict()
    TN = dict()
    TP = dict()
    FN = dict()
    temp = []
    la = []
    ROC = 0
    numclass = len(das)
    for c in lists:
        normal_class = c 
        
        new_model = MemAE()
        x, _ = new_model(x_test[0].reshape(1,32,32,3))
        new_model.load_weights('./MemAE_models/%s_%s_weights_%s.h5'%(id_name,id_model,normal_class))
        model = new_model 
       
            
        sc_ood = []
        for i in range(len(das[normal_class])):
            xx,_ = model(das[normal_class][i].reshape(1,32,32,3))
            score = mark(das[normal_class][i],xx)
            sc_ood.append(-score)
        
        test = np.asarray(sc_id[normal_class])
        ood = np.asarray(sc_ood)
    
        scores, mixed_labels = combine_inliners_outliers(test, ood)

        # fpr[c], tpr[c] = nums(scores, mixed_labels)
        # roc_auc[c] = auc(fpr[c], tpr[c])
        # print("%d th AUROC: %f"%(c,roc_auc[c]))
        
        lens = test.shape[0]
        FP[c],TN[c],TP[c],FN[c] = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    #     print(FP[i],TN[i],TP[i],FN[i])
        ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
        # print("%d th AUROC: %f"%(c,ROC))   
        temp.append(ROC)
        la.append(mixed_labels)
    for i in range(len(lists)):
        ROC += float(temp[i])       
    
    numbers_test = 0
    number_ood = 0
    for i in lists:
        numbers_test+= len(testimg[i])
        number_ood += len(das[i])
    numbers_all =  numbers_test +  number_ood
#     print('numbers_all is:',numbers_all)

    ROC_AVE = 0
    for i in range(len(lists)):
        ROC_AVE += float(len(la[i]/numbers_all)*temp[i])
    ROC_AVE = ROC_AVE/numbers_all

    NFP = 0
    NTN = 0
    NTP = 0
    NFN = 0
    for i in lists:
        NFP += FP[i]
    for i in lists:
        NTN += TN[i]
    for i in lists:
        NTP += TP[i]
    for i in lists:
        NFN += FN[i]
    print('The results of %s:\n'%(ds_name))
    #     print('fpr_at_95_tpr:', float(NFP) / float(NFP + NTN+ 1e-7) *100)
    #     print('detection_errror:',(1.0- float(NTP) / float(NTP + NFN + 1e-7)+ float(NFP) / float(NFP + NTN + 1e-7))/2*100)
    print('tnr_at_95_tpr:', float(NTN) / float(NFP + NTN+ 1e-7) *100)

    print('detection_acc:',(float(NTP) / float(NTP + NFN + 1e-7)+ float(NTN) / float(NFP + NTN + 1e-7))/2*100)
#     print("AUROC: ",ROC/(len(lists)+1)*100)
    print("Average AUROC: ",ROC_AVE*100)
    print("\n")

##Adversarial 

In [ ]:
testimg = np.load('datas_new/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)
sc_id = [[] for i in range(10)]
for c in range(10):
    normal_class = c 
    
    new_model = MemAE()
    x, _ = new_model(x_test[0].reshape(1,32,32,3))
    new_model.load_weights('./MemAE_models/%s_%s_weights_%s.h5'%(id_name,id_model,normal_class))
    model = new_model 
    
    x_test_normal = testimg[normal_class]
    for i in range(len(x_test_normal)):
        xx,_ = model(x_test_normal[i].reshape(1,32,32,3))
        score = mark(x_test_normal[i],xx)
        sc_id[c].append(-score)
        
np.save('./MemAE_datas_AD/testimgs_%s_%s.npy'%(id_model,id_name),sc_id)

In [ ]:
testimg = np.load('./datas_new/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)
sc_id = np.load('./MemAE_datas_AD/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)
ADV_DS_LIST = ("fgsm", "bim-a", "bim-b", "jsma", "cw-l2",'fooling_images')

for i, ds_name in enumerate(ADV_DS_LIST):
    ood = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('classdatas_new/'+ood+'.npy',allow_pickle=True)
    lists = []
    for i in range(10):
        if (len(das[i]) != 0):
            lists.append(i)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    FP = dict()
    TN = dict()
    TP = dict()
    FN = dict()
    temp = []
    la = []
    ROC = 0
    numclass = len(das)
    for c in lists:
        normal_class = c 
        
        new_model = MemAE()
        x, _ = new_model(x_test[0].reshape(1,32,32,3))
        new_model.load_weights('./MemAE_models/%s_%s_weights_%s.h5'%(id_name,id_model,normal_class))
        model = new_model 
        
        # sc_id = []
        # x_test_normal = testimg[normal_class]
        # for i in range(len(x_test_normal)):
        #     xx,_ = model(x_test_normal[i].reshape(1,28,28,1))
        #     score = mark(x_test_normal[i],xx)
        #     sc_id.append(-score)
        
            
        sc_ood = []
        for i in range(len(das[normal_class])):
            xx,_ = model(das[normal_class][i].reshape(1,32,32,3))
            score = mark(das[normal_class][i],xx)
            sc_ood.append(-score)
        
        test = np.asarray(sc_id[normal_class])
        ood = np.asarray(sc_ood)
    
        scores, mixed_labels = combine_inliners_outliers(test, ood)

        # fpr[c], tpr[c] = nums(scores, mixed_labels)
        # roc_auc[c] = auc(fpr[c], tpr[c])
        # print("%d th AUROC: %f"%(c,roc_auc[c]))
        
        lens = test.shape[0]
        FP[c],TN[c],TP[c],FN[c] = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    #     print(FP[i],TN[i],TP[i],FN[i])
        ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
        # print("%d th AUROC: %f"%(c,ROC))   
        temp.append(ROC)
        la.append(mixed_labels)
    for i in range(len(lists)):
        ROC += float(temp[i])       
    
    numbers_test = 0
    number_ood = 0
    for i in lists:
        numbers_test+= len(testimg[i])
        number_ood += len(das[i])
    numbers_all =  numbers_test +  number_ood
#     print('numbers_all is:',numbers_all)

    ROC_AVE = 0
    for i in range(len(lists)):
        ROC_AVE += float(len(la[i]/numbers_all)*temp[i])
    ROC_AVE = ROC_AVE/numbers_all

    NFP = 0
    NTN = 0
    NTP = 0
    NFN = 0
    for i in lists:
        NFP += FP[i]
    for i in lists:
        NTN += TN[i]
    for i in lists:
        NTP += TP[i]
    for i in lists:
        NFN += FN[i]
    print('The results of %s:\n'%(ds_name))
    #     print('fpr_at_95_tpr:', float(NFP) / float(NFP + NTN+ 1e-7) *100)
    #     print('detection_errror:',(1.0- float(NTP) / float(NTP + NFN + 1e-7)+ float(NFP) / float(NFP + NTN + 1e-7))/2*100)
    print('tnr_at_95_tpr:', float(NTN) / float(NFP + NTN+ 1e-7) *100)

    print('detection_acc:',(float(NTP) / float(NTP + NFN + 1e-7)+ float(NTN) / float(NFP + NTN + 1e-7))/2*100)
#     print("AUROC: ",ROC/(len(lists)+1)*100)
    print("Average AUROC: ",ROC_AVE*100)
    print("\n")